In [2]:
import pandas as pd
import numpy as np


In [7]:
x_arr = np.linspace(0,100)
df = pd.DataFrame({"x":x_arr,"y":[np.sin(x) for x in x_arr]})
df.to_csv("../../test_files/test_file_sin.csv",index=False)
df.head()

,x,y
0,0.000000,0.000000
1,2.040816,0.891559
2,4.081633,-0.807582
3,6.122449,-0.160045
4,8.163265,0.952552


In [11]:
import os

def get_extension_from_filename(filename:str):
    return os.path.splitext(filename)[-1][1:]

In [12]:
get_extension_from_filename("tmp.csv.txt")

'txt'